In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Load CSVs
temporal_df = pd.read_csv("temperature_temporal.csv")
spatial_df = pd.read_csv("temperature_spatial.csv")

# Merge on Segment_ID
df = pd.merge(temporal_df, spatial_df, on="Segment_ID")

# Separate temporal features (t0 to t299) and reshape to (samples, time_steps, features)
temporal_features = [col for col in df.columns if col.startswith("t")]
X_temporal = df[temporal_features].values.reshape(len(df), -1, 3)  # (samples, 100, 3)

# Normalize temporal data (optional: per-feature MinMax)
scaler_temp = MinMaxScaler()
X_temporal = X_temporal.reshape(-1, 3)
X_temporal = scaler_temp.fit_transform(X_temporal)
X_temporal = X_temporal.reshape(len(df), -1, 3)

# Spatial features (Distance, Elevation, Install_Year)
X_spatial = df[['Distance', 'Elevation', 'Install_Year']].values
scaler_spatial = MinMaxScaler()
X_spatial = scaler_spatial.fit_transform(X_spatial)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Defect_Label'])

# Train-test split
X_train_temp, X_test_temp, X_train_spatial, X_test_spatial, y_train, y_test = train_test_split(
    X_temporal, X_spatial, y, test_size=0.2, random_state=42, stratify=y
)

print("✅ Data processed:")
print(f"Temporal shape: {X_train_temp.shape}")
print(f"Spatial shape: {X_train_spatial.shape}")
print(f"Labels shape: {y_train.shape}")


✅ Data processed:
Temporal shape: (800, 100, 3)
Spatial shape: (800, 3)
Labels shape: (800,)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Concatenate
from sklearn.metrics import classification_report
import numpy as np

# --- Reuse preprocessed data from previous step ---
# Assumes: X_train_temp, X_test_temp, X_train_spatial, X_test_spatial, y_train, y_test, label_encoder

# === Model Definition ===

# Temporal Branch
input_temp = Input(shape=(100, 3), name='Temporal_Input')
x1 = LSTM(64, return_sequences=False)(input_temp)
x1 = Dropout(0.3)(x1)

# Spatial Branch
input_spatial = Input(shape=(3,), name='Spatial_Input')
x2 = Dense(32, activation='relu')(input_spatial)
x2 = Dropout(0.3)(x2)

# Combine both branches
combined = Concatenate()([x1, x2])
x = Dense(32, activation='relu')(combined)
output = Dense(len(np.unique(y_train)), activation='softmax')(x)

model = Model(inputs=[input_temp, input_spatial], outputs=output)

# === Compile & Train ===
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    [X_train_temp, X_train_spatial], y_train,
    validation_data=([X_test_temp, X_test_spatial], y_test),
    epochs=20, batch_size=16, verbose=2
)

# === Evaluate ===
loss, acc = model.evaluate([X_test_temp, X_test_spatial], y_test, verbose=0)
print(f"\n✅ Test Accuracy: {acc:.4f}")

# === Classification Report ===
y_pred = model.predict([X_test_temp, X_test_spatial])
y_pred_labels = np.argmax(y_pred, axis=1)

print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred_labels, target_names=label_encoder.classes_))

# === Model Summary ===
model.summary()


Epoch 1/20
50/50 - 1s - 30ms/step - accuracy: 0.5725 - loss: 1.1757 - val_accuracy: 0.5750 - val_loss: 1.1296
Epoch 2/20
50/50 - 0s - 9ms/step - accuracy: 0.5775 - loss: 1.1330 - val_accuracy: 0.5750 - val_loss: 1.1275
Epoch 3/20
50/50 - 0s - 9ms/step - accuracy: 0.5775 - loss: 1.1312 - val_accuracy: 0.5750 - val_loss: 1.1249
Epoch 4/20
50/50 - 0s - 9ms/step - accuracy: 0.5775 - loss: 1.1288 - val_accuracy: 0.5750 - val_loss: 1.1272
Epoch 5/20
50/50 - 0s - 9ms/step - accuracy: 0.5775 - loss: 1.1266 - val_accuracy: 0.5750 - val_loss: 1.1253
Epoch 6/20
50/50 - 0s - 9ms/step - accuracy: 0.5775 - loss: 1.1240 - val_accuracy: 0.5750 - val_loss: 1.1263
Epoch 7/20
50/50 - 0s - 9ms/step - accuracy: 0.5775 - loss: 1.1187 - val_accuracy: 0.5750 - val_loss: 1.1272
Epoch 8/20
50/50 - 0s - 9ms/step - accuracy: 0.5775 - loss: 1.1311 - val_accuracy: 0.5750 - val_loss: 1.1286
Epoch 9/20
50/50 - 0s - 9ms/step - accuracy: 0.5775 - loss: 1.1233 - val_accuracy: 0.5750 - val_loss: 1.1242
Epoch 10/20
50/50 

c:\Users\Haard\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Haard\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Haard\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Temporal_Input      │ (None, 100, 3)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Spatial_Input       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 64)        │     17,408 │ Temporal_Input[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │        128 │ Spatial_Input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 64)        │          0 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 32)        │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 96)        │          0 │ dropout_5[0][0],  │
│ (Concatenate)       │                   │            │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32)        │      3,104 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 4)         │        132 │ dense_6[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 62,318 (243.43 KB)

 Trainable params: 20,772 (81.14 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 41,546 (162.29 KB)